In [14]:
import os
import re
import pandas as pd
import numpy as np
from tensorflow import keras
import tensorflow as tf

In [15]:
model_path = r'C:\Users\91999\OneDrive\Desktop\DRR project\general\trained_model\A2\DRR_MTL_3c_7c_250e'

In [16]:
base_path = r'D:\new microscope'

In [17]:
output_path = r'C:\Users\91999\OneDrive\Desktop\ViT Outputs\ViT micro E 3c 7c.xlsx'

In [18]:
# Define image dimensions and load the model
img_dim = 288

from tensorflow_addons.optimizers import AdamW

# Register the custom object
with tf.keras.utils.custom_object_scope({'AdamW': AdamW}):
    model = tf.keras.models.load_model(model_path)
#model =  tf.keras.models.load_model(model_path)

In [19]:
# Extract image file paths
image_file_paths = []
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith((".png", ".PNG", ".jpeg")):  # Add other image extensions if necessary
            image_file_paths.append(os.path.join(root, file))


In [20]:
# Load test dataset without shuffling to maintain order
test_ds = keras.utils.image_dataset_from_directory(
    base_path,
    image_size=(img_dim, img_dim),
    label_mode='int',
    batch_size=1000,
    shuffle=False  # Ensure no shuffling to keep image file order intact
)

Found 710 files belonging to 2 classes.


In [21]:
class_names = test_ds.class_names

In [22]:
resize_and_rescale = keras.Sequential([
  #layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
  keras.layers.experimental.preprocessing.Rescaling(1./255)
])

In [23]:
data_augmentation = keras.Sequential([
    keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    keras.layers.experimental.preprocessing.RandomRotation(0.1),
    #keras.layers.experimental.preprocessing.RandomFlip("vertical"),
])

In [24]:
# Function to prepare dataset
AUTOTUNE = tf.data.AUTOTUNE
def prepare(ds, augment=False):
    def resize_and_rescale(image, label):
        image = tf.image.resize(image, [img_dim, img_dim])
        image = tf.cast(image, tf.float32) / 255.0
        return image, label
    
    # Apply resizing and rescaling
    ds = ds.map(lambda x, y: (resize_and_rescale(x, y)), num_parallel_calls=AUTOTUNE)
    
    # Apply data augmentation if specified
    if augment:
        data_augmentation = tf.keras.Sequential([
            tf.keras.layers.RandomFlip("horizontal_and_vertical"),
            tf.keras.layers.RandomRotation(0.2)
        ])
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)
        
    # Use buffered prefetching
    return ds.prefetch(buffer_size=AUTOTUNE)


In [25]:
# Prepare the test dataset
test_ds = prepare(test_ds)


In [26]:
# Separate the images and labels
ts_image_batch,tr_label_batch = next(iter(test_ds))
ts_image_batch = np.asarray(ts_image_batch)

In [ ]:
preds = model.predict(ts_image_batch)


In [ ]:
# Extract predicted class indices and scores
y_pred_3c = []
for i in preds[0]:
    y_pred_3c = np.append(y_pred_3c,np.argmax(i))

y_pred_scores = []
for i in preds[1]:
    y_pred_scores = np.append(y_pred_scores,np.argmax(i))

In [ ]:
actual_classes = [class_names[label] for label in tr_label_batch]
L = ['CONTROL','NON DRR','PATHOGEN']
y_pred_classes = [L[int(label)] for label in y_pred_3c]

In [ ]:
# Extract image file names using regex
#image_file_names = [re.match(file_name_pattern, file_path).group(1) for file_path in image_file_paths[:len(ts_label_batch)]]
image_file_names = [os.path.basename(file_path) for file_path in image_file_paths[:len(ts_image_batch)]]

In [ ]:
print(len(image_file_names), len(y_pred_classes), len(y_pred_scores))

In [ ]:
image_file_paths = []
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith((".png", ".jpg", ".jpeg")):  # Add other image extensions if necessary
            image_file_paths.append(os.path.join(root, file))

print("Total images found:", len(image_file_paths))


In [ ]:
# Create a DataFrame with Image Name, Predicted Class, Predicted Score, Actual Class
results_df = pd.DataFrame({
    'Folder': actual_classes,
    'Image Name': image_file_names,
    'Predicted Class': y_pred_classes,
    'Predicted Score': y_pred_scores
})


In [ ]:
# Save the DataFrame to an Excel file with a different name

results_df.to_excel(excel_writer=output_path, index=False)

print(f"Results have been saved to {output_path}")


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 2)
display(results_df)